In [1]:
from scipy import signal
from scipy.signal import find_peaks
import numpy as np
import statistics as st

C:\Users\User\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def functionswingtime(idxTO, idxHS, sample_frequency, time):

    swingTO = np.array([])
    swingHS = np.array([])
        
    for i in range(len(idxHS)):
        lastTO = np.argwhere(idxTO < idxHS[i])
        if len(lastTO) > 0:
            lastTO = lastTO[-1][0]
            swingTO = np.append(swingTO, idxTO[lastTO])
            swingHS = np.append(swingHS, idxHS[i])
         
    return  swingTO, swingHS

In [3]:
def functionstancetime(idxTO, idxHS, sample_frequency, time):
    
    stanceTO = np.array([])
    stanceHS = np.array([])
        
    for i in range(len(idxTO)):
        lastHS = np.argwhere(idxHS < idxTO[i])
        if len(lastHS) > 0:
            lastHS = lastHS[-1][0]
            stanceTO = np.append(stanceTO, idxTO[i])
            stanceHS = np.append(stanceHS, idxHS[lastHS])
    
    return  stanceTO, stanceHS



In [4]:
def accPeaks(fcacc, time,idxHS):
    # Initialize lists for first higher and lower peaks between heel strikes
    first_higher_peaks_times = []
    first_higher_peaks_values = []
    first_lower_peaks_times = []
    first_lower_peaks_values = []

    # Loop through each pair of consecutive heel strikes
    for i in range(len(idxHS) - 1):
        start_idx = idxHS[i]
        end_idx = idxHS[i + 1]

        # Extract the segment of the signal between heel strikes
        segment = fcacc[start_idx:end_idx]
        segment_time = time.iloc[start_idx:end_idx]

#         Detect all higher peaks (local maxima)
        higher_peaks = find_peaks(segment, prominence=3)[0]
        # Detect all lower peaks (local minima) by inverting the signal
        lower_peaks = find_peaks(-segment, prominence=3)[0]

#         # Check if higher_peaks and lower_peaks are not empty
        if higher_peaks.size > 0 and lower_peaks.size > 0:
            if segment[higher_peaks[0]] >segment[lower_peaks[0]]:
                first_idx = higher_peaks[0]
                next_peak_indices = lower_peaks[lower_peaks > first_idx]
                if next_peak_indices.size > 0:
                    first_higher_peaks_times.append(segment_time.iloc[first_idx])
                    first_higher_peaks_values.append(segment[first_idx])
                    first_lower_peaks_times.append(segment_time.iloc[next_peak_indices[0]])
                    first_lower_peaks_values.append(segment[next_peak_indices[0]])
            else:
                first_idx = lower_peaks[0]
                next_peak_indices = higher_peaks[higher_peaks > first_idx]
                if next_peak_indices.size > 0:
                    first_higher_peaks_times.append(segment_time.iloc[next_peak_indices[0]])
                    first_higher_peaks_values.append(segment[next_peak_indices[0]])
                    first_lower_peaks_times.append(segment_time.iloc[first_idx])
                    first_lower_peaks_values.append(segment[first_idx])
    return first_lower_peaks_times, first_lower_peaks_values, first_higher_peaks_times, first_higher_peaks_values
